## import statements

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as sp

## define some helpful functions

In [3]:
def factorial(x):
    return sp.factorial(x, exact=True)

def multiplicityEinstein(N,q):
    return factorial(q+N-1)//factorial(q)//factorial(N-1)

def logArray(array):
    import math as math
    return [math.log(x) for x in array]

## Two Einstein Solids
### 3 particles each and 6 energy units

In [4]:
table1 = pd.DataFrame({'q_A':range(0, 6+1, 1),'q_B':range(6,0-1,-1)})
table1['multi_A'] = [multiplicityEinstein(3,i) for i in table1['q_A']]
table1['multi_B'] = [multiplicityEinstein(3,i) for i in table1['q_B']]
table1['multi_total'] = table1['multi_A']*table1['multi_B']

table1

,q_A,q_B,multi_A,multi_B,multi_total
0,0,6,1,28,28
1,1,5,3,21,63
2,2,4,6,15,90
3,3,3,10,10,100
4,4,2,15,6,90
5,5,1,21,3,63
6,6,0,28,1,28


#### how many total microstates from the above situation?

First just add up the microstate from each macrostate

In [5]:
table1['multi_total'].sum()

462

Alternatively, treat the two Einstein solids as one solid with the combined number of particles

In [6]:
multiplicityEinstein(6,6)

462

#### most likely macrostate
just sort the table for the macrostate with the highest total multiplicity.

In [7]:
table1.sort_values('multi_total', ascending=False).iloc[0]

q_A              3
q_B              3
multi_A         10
multi_B         10
multi_total    100
Name: 3, dtype: int64

## Now prepare a function to scale this up to large numbers

In [19]:
def multiTable(N_a, N_b, q):
    df = pd.DataFrame({'q_A':range(0, q+1, 1),'q_B':range(q, 0-1,-1)})
    df['multi_A'] = [multiplicityEinstein(N_a, i) for i in df['q_A']]
    df['multi_B'] = [multiplicityEinstein(N_b, i) for i in df['q_B']]
    df['multi_total'] = df['multi_A']*df['multi_B']
    return df

### Try with Na = 300, Nb = 200, q = 100

In [20]:
table2 = multiTable(300, 200, 100)

In [21]:
fig0 = plt.figure()
ax0 = fig0.add_subplot(111)

ax0.bar(table2['q_A'], table2['multi_total'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 101 artists>

Make these numbers floats instead of integers so we can read them better. Note that this will break for numbers much larger than this.

In [22]:
table2 = table2.astype(float)

Total multiplicity of all macrostates:

In [23]:
total = table2['multi_total'].sum()
total

9.261760158884879e+115

Total mulitplicity of all states as a combined Einstein solid.

In [24]:
float(multiplicityEinstein(500,100))

9.261760158884879e+115

Most probable macrostate

In [25]:
table2.sort_values('multi_total', ascending=False).iloc[0]

q_A             6.000000e+01
q_B             4.000000e+01
multi_A         1.303375e+69
multi_B         5.268097e+45
multi_total    6.866305e+114
Name: 60, dtype: float64

Multiplicity of the most common macrostate:

In [26]:
multi_max = table2['multi_total'].sort_values(ascending=False).iloc[0]
multi_max

6.866305444480905e+114

### Now to add in the Entropy

In [27]:
table2['S_a'] = logArray(table2['multi_A'])
table2['S_b'] = logArray(table2['multi_B'])
table2['S_total'] = logArray(table2['multi_total'])

In [28]:
table2

,q_A,q_B,multi_A,multi_B,multi_total,S_a,S_b,S_total
0,0.0,100.0,1.000000e+00,2.772168e+81,2.772168e+81,0.000000,187.529022,187.529022
1,1.0,99.0,3.000000e+02,9.271464e+80,2.781439e+83,5.703782,186.433749,192.137531
2,2.0,98.0,4.515000e+04,3.080117e+80,1.390673e+85,10.717746,185.331775,196.049521
3,3.0,97.0,4.545100e+06,1.016335e+80,4.619344e+86,15.329560,184.223010,199.552571
4,4.0,96.0,3.442913e+08,3.330557e+79,1.146682e+88,19.656999,183.107362,202.764361
5,5.0,95.0,2.093291e+10,1.083842e+79,2.268798e+89,23.764589,181.984735,205.749323
6,6.0,94.0,1.064090e+12,3.502212e+78,3.726667e+90,27.693141,180.855032,208.548173
7,7.0,93.0,4.651592e+13,1.123576e+78,5.226419e+91,31.470816,179.718154,211.188970
8,8.0,92.0,1.785049e+15,3.578514e+77,6.387821e+92,35.118222,178.574000,213.692222
9,9.0,91.0,6.108833e+16,1.131351e+77,6.911237e+93,38.651097,177.422465,216.073562


#### Plotting Entropy

In [29]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)

ax1.plot(table2['q_A'], table2['S_a'], label='S_a')
ax1.plot(table2['q_A'], table2['S_b'], label='S_b')
ax1.plot(table2['q_A'], table2['S_total'], label='S_total')

# labels
ax1.set_ylabel('Entropy')
ax1.set_xlabel('q_A')

#legend
ax1.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …